In [1]:
import time
import numpy as np
from godotbridge import GodotBridge

class Alpha(GodotBridge):

    def stop(self):
        self.set("set_vr", 0)
        self.set("set_vl", 0)

    def move(self, right_vel, left_vel):
        self.set("set_vr", right_vel)
        self.set("set_vl", left_vel)
    def set_pose(self, x, z, a):
        self.set("set_pose", x, z, a)
    def get_pose(self):
        return self.get("get_pose")
    def is_running(self) -> bool:
        return self.get("is_running")
    
class Timer:
    def start(self, delay):
        self.start_time = time.time()
        self.delay = delay
    def is_elapsed(self):
        return (time.time() - self.start_time) > self.delay
    
class StateMachine:
    def __init__(self, states) -> None:
        self.states = states
        self.activate = False
    
    def start(self, init_state) -> None:
        self.running = True
        self.state = init_state
        self.once = True
        self.new_once = False

    def update(self):
        self.states[self.state](self.once)
        if self.new_once:
            self.once = True
            self.new_once = False
        else:
            self.once = False

    def set_state(self, name) -> None:
        self.state = name
        self.new_once = True

Connexion avec le robot Alpha

Avance du robot

In [2]:
# alpha = Alpha(4242)
# left_vel = 5
# right_vel = 2
# alpha.move(right_vel, left_vel)
# time.sleep(2)
# alpha.stop()

Placement du robot

In [3]:
# alpha = Alpha(4242)
# x = 0
# z = 0
# a = 45
# alpha.set_pose(x, z, a)

Essai de tracé de trajectoire

In [5]:
# State functions
def pose(once):
    if once:
        # print("init")
        alpha.set_pose(0.1,0,0)
    else:
        sm.set_state("forward")

def forward(once):
    if once:
        # print("forward")
        alpha.move(5, 5)
        timer.start(1)
    else:
        if timer.is_elapsed():
            sm.set_state("turn_left")

def backward(once):
    pass

def turn_left(once):
    if once:
        # print("turn left")
        alpha.move(4,-4)
        timer.start(1)
    if timer.is_elapsed():
        sm.set_state("forward")

def stop(once):
    if once:
        # print("stop")
        alpha.move(0, 0)
    else:
        sm.running = False

states = {
    "init": pose,
    "forward": forward,
    "turn_left": turn_left,
    "stop": stop,
}

timer = Timer()
sm = StateMachine(states)
alpha = Alpha(4242)
if alpha.is_running():
    sm.start("init")

    while sm.running and alpha.is_running():
        sm.update()


print("Game is not running!")
alpha.stop()

Game is not running!
